Initially, I relied on annotations from [Pfam](ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.regions.uniprot.tsv.gz) and [Uniprot](ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/docs/pdbtosp.txt) (downloaded using the bash script [get_data.sh](./data/get_data.sh)), and focused on the [classic zinc finger domain](https://pfam.xfam.org/family/PF00096).

In [122]:
from Bio import SeqIO
import gzip
import os
import pandas as pd
import pickle
import re
import subprocess as sp
import tarfile
from urllib import request

# Initialize
pfam_accession = "PF00096"
prosite_id = "PS50157"
CTCF = "P49711"

#++++++++++++++++#
# Pickle recipes #
#++++++++++++++++#

# Adapted from "Load Faster in Python With Compressed Pickles"
# https://medium.com/better-programming/load-fast-load-big-with-compressed-pickles-5f311584507e

def save_pickle(file_name, pkl):
    """
    Saves a pickle.
    """

    with gzip.open(file_name, "wb") as f:
        pickle.dump(pkl, f)

def load_pickle(file_name):
    """
    Loads and returns a pickle.
    """

    with gzip.open(file_name, "rb") as f:
        pkl = pickle.load(f)

    return(pkl)

I extracted the UniProt human proteome reference sequences (only one per gene) as [Seq](https://biopython.org/wiki/Seq) objects (*i.e.* `human_seqs`)...

In [124]:
# Initialize
human_seqs_pickle = "./pkl/human_sequences.pkl.gz"

if not os.path.exists(human_seqs_pickle):

    # Initialize
    human_seqs = {}
    pattern = re.compile("^\w{2}\|(\w+)")

    with gzip.open("./data/UP000005640_9606.fasta.gz", "rt") as f:
        for seq_record in SeqIO.parse(f, "fasta"):
            uniacc = pattern.match(seq_record.id)
            human_seqs.setdefault(uniacc.group(1), seq_record.seq)

    save_pickle(human_seqs_pickle, human_seqs)

else:

    human_seqs = load_pickle(human_seqs_pickle)

# Sanity check:
# According to UniProt's human reference proteome (https://www.uniprot.org/proteomes/UP000005640),
# the gene count is 20,595
print(len(human_seqs))

20595


... and all zinc fingers, according to Pfam, mapped to human (and other UniProt) proteins (*i.e.* `uprot2pfam`).

In [125]:
# Initialize
uprot2pfam_pickle = "./pkl/uniprot_to_pfam.pkl.gz"

if not os.path.exists(uprot2pfam_pickle):

    # Initialize
    uprot2pfam = {}

    with gzip.open("./data/Pfam-A.regions.uniprot.tsv.gz", "rt") as f:
         for chunk in pd.read_csv(f, encoding="utf-8", sep="\t", chunksize=1024):
            for index, row in chunk.iterrows():
                uniacc, seq_version, crc64, md5, pfam_acc, start, end = row.tolist()
                if pfam_acc == pfam_accession:
                    uprot2pfam.setdefault(uniacc, set())
                    uprot2pfam[uniacc].add(tuple([start, end]))

    save_pickle(uprot2pfam_pickle, uprot2pfam)

else:

    uprot2pfam = load_pickle(uprot2pfam_pickle)

# Sanity checks:
# The total number of C2H2-zf proteins in UniProt is?
print("The total number of C2H2-zf proteins in UniProt is: %s" % len(uprot2pfam))
# According to "The Human Transcription Factors" (PMID: 29425488), the total C2H2-zf TFs is 747
n = len(set(uprot2pfam.keys()).intersection(set(human_seqs.keys())))
print("For human, the number is: %s" % n)
# According to "Analysis of the vertebrate insulator protein CTCF-binding sites in the human genome" (PMID: 17382889),
# the total number of C2H2-zf domains for CTCF is 11
print("And for CTCF: %s" % len(uprot2pfam[CTCF]))

The total number of C2H2-zf proteins in UniProt is: 160662
For human, the number is: 678
And for CTCF: 6


Together, the fact that both i) the number of human C2H2-zf proteins and ii) CTCF C2H2-zf domains are lower than expected suggests that there might be something wrong with the Pfam thresholding used by UniProt.

Instead, I switched to PROSITE domain annotations by the [zinc finger C2H2 type domain profile](https://prosite.expasy.org/PS50157), which [match CTCF better](https://www.uniprot.org/uniprot/P49711).

The file [uniprot2prosite.tab.gz](./data/uniprot2prosite.tab.gz) is obtained from [UniProt](https://www.uniprot.org/database/DB-0084) by customizing the output to `Entry`, `Gene names`, `Organism` and `PROSITE`.

I extracted all C2H2-zf proteins from UniProt, according to PROSITE (*i.e.* `c2h2_zfs`).

In [126]:
# Initialize
c2h2_zf_pickle = "./pkl/c2h2_zf.pkl.gz"
pattern = re.compile(prosite_id)

if not os.path.exists(c2h2_zf_pickle):

    # Initialize
    c2h2_zfs = {}

    with gzip.open("./data/uniprot2prosite.tab.gz", "rt") as f:
         for chunk in pd.read_csv(f, encoding="utf-8", sep="\t", chunksize=1024):
            for index, row in chunk.iterrows():
                uniacc, gene_name, organism, prosites = row.tolist()
                if pattern.search(prosites):
                    c2h2_zfs.setdefault(organism, set())
                    c2h2_zfs[organism].add(uniacc)

    save_pickle(c2h2_zf_pickle, c2h2_zfs)

else:

    c2h2_zfs = load_pickle(c2h2_zf_pickle)    

# Sanity checks:
# The total number of C2H2-zf proteins in UniProt is?
c2h2_zf_set = set()
for f in c2h2_zfs:
    c2h2_zf_set.update(c2h2_zfs[f])
print("The total number of C2H2-zf proteins in UniProt is: %s" % len(c2h2_zf_set))
# According to "The Human Transcription Factors" (PMID: 29425488), the total C2H2-zf TFs is 747
n = len(c2h2_zfs["Homo sapiens (Human)"].intersection(set(human_seqs.keys())))
print("And for human, the number is: %s" % n)

The total number of C2H2-zf proteins in UniProt is: 353167
And for human, the number is: 762


Then, I retrieved the sequences (only one per gene) of all C2H2-zf proteins from the UniProt [reference proteomes of eukaryotes](ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Reference_Proteomes_2020_02.tar.gz).

In [127]:
# Initialize
c2h2_zf_seqs_pickle = "./pkl/c2h2_zf_sequences.pkl.gz"

if not os.path.exists(c2h2_zf_seqs_pickle):

    # Initialize
    c2h2_zf_seqs = {}
    dummy_dir = "./data/"
    pattern = re.compile("Eukaryota/UP\d+_\d+.fasta.gz")
    pattern2 = re.compile("^\w{2}\|(\w+)")
    reference_proteomes_fasta = "./data/Reference_Proteomes_2020_01.tar.gz"

    with tarfile.open(reference_proteomes_fasta, "r:gz") as tar:
        for member in tar.getmembers():
            # Skip organisms other than eukaryotes
            if not member.name.startswith("Eukaryota"):
                continue
            # Skip files other than reference proteomes
            if pattern.search(member.name):
                reference_proteome_file = os.path.join(dummy_dir, member.name) 
                if not os.path.exists(reference_proteome_file):
                    tar.extract(member, path=dummy_dir)
                with gzip.open(reference_proteome_file, "rt") as f:
                    for seq_record in SeqIO.parse(f, "fasta"):
                        uniacc = pattern2.match(seq_record.id)
                        if uniacc.group(1) in c2h2_zf_set:
                            c2h2_zf_seqs.setdefault(uniacc.group(1), str(seq_record.seq))

    save_pickle(c2h2_zf_seqs_pickle, c2h2_zf_seqs)

else:

    c2h2_zf_seqs = load_pickle(c2h2_zf_seqs_pickle)

# Total C2H2-zf transcription factors in reference proteomes of eukaryotes is?
print("The total number of C2H2-zf proteins in UniProt reference proteomes of eukaryotes is: %s" % len(c2h2_zf_seqs))

The total number of C2H2-zf proteins in UniProt reference proteomes of eukaryotes is: 186703


Finally, I scanned the retrieved sequences with the [zinc finger C2H2 type domain profile](https://prosite.expasy.org/PS50157) from PROSITE and, for each protein, I stored the positions of each match (*i.e.* `uprot2prosite`).

In [128]:
# Initialize
uprot2prosite_pickle = "./pkl/uniprot_to_prosite.pkl.gz"

def scan_prosite(uniacc, uniacc2seqs):

    # Initialize
    matches = set()
    fasta_file = "./data/%s.fasta" % uniacc

    if os.path.exists(fasta_file):
        os.remove(fasta_file)

    with open(fasta_file, "w") as o:
        o.write(">%s\n%s\n" % (uniacc, uniacc2seqs[uniacc]))

    # Run PROSITE
    cmd = "perl %s/ps_scan.pl -d %s/prosite.dat -p %s %s" % (prosite_dir, prosite_dir, prosite_id, fasta_file)
    p = sp.run([cmd], shell=True, stdout=sp.PIPE, stderr=sp.PIPE)
    for line in p.stdout.decode("utf-8").split("\n"):
        if not line.startswith(">"):
            match = re.findall("\S+", line)
            if match:
                matches.add(tuple([match[0], match[2]]))

    os.remove(fasta_file)

    return(uniacc, matches)

if not os.path.exists(uprot2prosite_pickle):

    # Initialize
    uprot2prosite = {}
    c2h2_zf_prosite_matches = "./data/c2h2_zf_prosite_matches.txt"

    if not os.path.exists(c2h2_zf_prosite_matches):

        # Initialize
        prosite_dir = "/space/home/oriol/Programs/ps_scan"
        c2h2_zf_seqs_fasta = "./data/c2h2_zf_sequences.fasta"

        if not os.path.exists(c2h2_zf_seqs_fasta):
            with open(c2h2_zf_seqs_fasta, "w") as o:
                for uniacc in c2h2_zf_seqs:
                    o.write(">%s\n%s\n" % (uniacc, c2h2_zf_seqs[uniacc]))

        # Run PROSITE
        cmd = "perl %s/ps_scan.pl -d %s/prosite.dat -p %s %s > %s" % (prosite_dir, prosite_dir, prosite_id,
            c2h2_zf_seqs_fasta, c2h2_zf_prosite_matches)
        os.system(cmd)

    with open(c2h2_zf_prosite_matches) as f:
        for line in f:
            if line.startswith(">"):
                m = re.search(">(\S+) :", line)
                uniacc = m.group(1)
                uprot2prosite.setdefault(uniacc, set())
            else:
                m = re.search("(\d+) - (\d+)", line)
                if m:
                    uprot2prosite[uniacc].add(tuple([m.group(1), m.group(2)]))

    save_pickle(uprot2prosite_pickle, uprot2prosite)

else:

    uprot2prosite = load_pickle(uprot2prosite_pickle)

# Sanity checks:
# The total number of C2H2-zf domains in UniProt reference proteomes of eukaryotes is?
n = sum([len(uprot2prosite[uniacc]) for uniacc in uprot2prosite])
print("The total number of C2H2-zf domains in UniProt reference proteomes of eukaryotes is: %s" % n)
# The total number of human C2H2-zf domains in UniProt reference proteomes of eukaryotes is?
n = sum([len(uprot2prosite[uniacc]) for uniacc in set(uprot2prosite.keys()).intersection(set(human_seqs.keys()))])
print("For human, the number is: %s" % n)
# According to "Analysis of the vertebrate insulator protein CTCF-binding sites in the human genome" (PMID: 17382889),
# the total number of C2H2-zf domains for CTCF is 11
print("And for CTCF: %s" % len(uprot2prosite[CTCF]))

The total number of C2H2-zf domains in UniProt reference proteomes of eukaryotes is: 963553
For human, the number is: 7180
And for CTCF: 11


The PROSITE results are more in line with my expectations. Hence, I will focus on the PROSITE-based C2H2-zf proteins and domains.

The file [uniprot2pdb.tab.gz](./data/uniprot2pdb.tab.gz) is obtained from [UniProt](https://www.uniprot.org/database/DB-0070) by customizing the output to `Entry`, `Gene names`, `Organism` and `PDB`.

I extracted all PDB accessions associated with C2H2-zf proteins from UniProt.

In [129]:
# Initialize
c2h2_zf_pdb_pickle = "./pkl/c2h2_zf_pdb.pkl.gz"

if not os.path.exists(c2h2_zf_pdb_pickle):

    # Initialize
    c2h2_zfs_pdbs = {}

    with gzip.open("./data/uniprot2pdb.tab.gz", "rt") as f:
         for chunk in pd.read_csv(f, encoding="utf-8", sep="\t", chunksize=1024):
            for index, row in chunk.iterrows():
                uniacc, gene_name, organism, pdb_string = row.tolist()
                if organism in c2h2_zfs:
                    if uniacc in c2h2_zfs[organism]:
                        pdb_list = pdb_string.split(";")
                        p = pdb_list.pop()
                        c2h2_zfs_pdbs.setdefault(uniacc, pdb_list)

    save_pickle(c2h2_zf_pdb_pickle, c2h2_zfs_pdbs)

else:

    c2h2_zfs_pdbs = load_pickle(c2h2_zf_pdb_pickle) 

# The total number of C2H2-zf proteins with a 3D structure in the PDB is?
print("The total number of C2H2-zf proteins with a 3D structure in the PDB is: %s" % len(c2h2_zfs_pdbs))
# The total number of human C2H2-zf proteins with a 3D structure in the PDB is?
n = set(c2h2_zfs_pdbs.keys()).intersection(human_seqs.keys())
print("For human, the number is: %s" % len(n))

The total number of C2H2-zf proteins with a 3D structure in the PDB is: 136
For human, the number is: 91


Then, I retrieved all protein-DNA accessions from the PDB.

In [130]:
# Initialize
pdb_protein_dna_pickle = "./pkl/pdb_protein_dna.pkl.gz"

if not os.path.exists(pdb_protein_dna_pickle):

    # Initialize
    pattern = re.compile("\w{4}")
    pdb_protein_dna = set()
    cwd = os.getcwd()
    url = "http://www.rcsb.org/pdb/rest/search"
    xml = "".join(['<?xml version="1.0" encoding="UTF-8"?>',
                   '<orgPdbQuery>',
                   '<queryType>org.pdb.query.simple.ChainTypeQuery</queryType>',
                   '<description>Chain Type Search : Contains Protein=Y, Contains DNA=Y, Contains RNA=I, Contains DNA/RNA Hybrid=I</description>',
                   '<containsProtein>Y</containsProtein>',
                   '<containsDna>Y</containsDna>',
                   '<containsRna>I</containsRna>',
                   '<containsHybrid>I</containsHybrid>',
                   '</orgPdbQuery>'])

    r = request.Request(url, data=xml.encode("utf-8"))
    for line in request.urlopen(r):
        pdb = line.decode("utf-8").strip()
        if pattern.search(pdb):
            pdb_protein_dna.add(pdb)

    save_pickle(pdb_protein_dna_pickle, pdb_protein_dna)

else:

    pdb_protein_dna = load_pickle(pdb_protein_dna_pickle)

# The total number of protein-DNA complexes in the PDB is?
print("The total number of protein-DNA complexes in the PDB is: %s" % len(pdb_protein_dna))
# The total number of protein-DNA complexes of C2H2-zf proteins in the PDB is?
n = 0
for uniacc in c2h2_zfs_pdbs:
    if set(c2h2_zfs_pdbs[uniacc]).intersection(pdb_protein_dna):
        n += 1
        if uniacc == CTCF:
            CTCF_set = sorted(set(c2h2_zfs_pdbs[uniacc]).intersection(pdb_protein_dna))
print("For C2H2-zf proteins, the number is: %s" % n)
# The total number of protein-DNA complexes of human C2H2-zf proteins in the PDB is?
n = 0
for uniacc in c2h2_zfs_pdbs:
    if uniacc not in human_seqs:
        continue
    if set(c2h2_zfs_pdbs[uniacc]).intersection(pdb_protein_dna):
        n += 1
print("And for human: %s" % n)
print("* CTCF has %s protein-DNA PDB complexes: %s;" % (len(CTCF_set), ";".join(CTCF_set)))

The total number of protein-DNA complexes in the PDB is: 5587
For C2H2-zf proteins, the number is: 23
And for human: 12
* CTCF has 12 protein-DNA PDB complexes: 5K5H;5K5I;5K5J;5K5L;5KKQ;5T00;5T0U;5UND;5YEF;5YEG;5YEH;5YEL;


Now here was the tricky part: How many (what percentage of) C2H2-zf domains are covered in the PDB?

To answer this question, I used the [pdb2uniprot](https://github.com/mgalardini/pdb2uniprot) package developed by [Marco Galardini](https://github.com/mgalardini).

First of all, I generated a list of PDB accessions associated with C2H2-zf proteins with which to run pdb2uniprot.

In [131]:
# Initialize
c2h2_zf_pdb_file = "./data/c2h2_zf_pdb.txt"

if not os.path.exists(c2h2_zf_pdb_file):

    with open(c2h2_zf_pdb_file, "w") as o:
        for uniacc in c2h2_zfs_pdbs:
            # i.e. lower is required by pdb2uniprot
            o.write("%s\n" % "\n".join([pdb.lower() for pdb in c2h2_zfs_pdbs[uniacc]]))

The file [c2h2_zf_pdb2uniprot]("./data/c2h2_zf_pdb2uniprot.tsv.gz") stores the results from pdb2uniprot:

```
$ zless c2h2_zf_pdb2uniprot.tsv.gz | head
pdb id	pdb chain	pdb residue	pdb position	uniprot id	uniprot residue	uniprot position
6u04	A	A	1	P84243	A	2
6u04	A	R	2	P84243	R	3
6u04	A	T	3	P84243	T	4
6u04	A	K	null	P84243	K	5
6u04	A	Q	null	P84243	Q	6
6u04	A	T	null	P84243	T	7
6u04	A	A	null	P84243	A	8
6u04	A	R	null	P84243	R	9
6u04	A	K	null	P84243	K	10
```

Then, for each C2H2-zf PDB structure, I extracted the residues of the UniProt C2H2-zf protein covered. 

In [132]:
# Initialize
c2h2_zf_pdb2uniprot_pickle = "./pkl/c2h2_zf_pdb2uniprot.pkl.gz"

if not os.path.exists(c2h2_zf_pdb2uniprot_pickle):

    # Initialize
    c2h2_zf_pdb2uniprot = {}

    with gzip.open("./data/c2h2_zf_pdb2uniprot.tsv.gz", "rt") as f:
        for line in f:
            if line.startswith("pdb id"):
                continue
            pdb, pdb_chain, pdb_res, pdb_pos, uniacc, uni_res, uni_pos = line.strip().split("\t")
            if pdb_res != "null":
                pdb2uniprot = tuple([pdb, uniacc])
                c2h2_zf_pdb2uniprot.setdefault(pdb2uniprot, {})
                c2h2_zf_pdb2uniprot[pdb2uniprot].setdefault(pdb_chain, [])
                c2h2_zf_pdb2uniprot[pdb2uniprot][pdb_chain].append(int(uni_pos))

    save_pickle(c2h2_zf_pdb2uniprot_pickle, c2h2_zf_pdb2uniprot)

else:

    c2h2_zf_pdb2uniprot = load_pickle(c2h2_zf_pdb2uniprot_pickle)

# Sanity checks:
# The total number of protein-DNA complexes in the PDB for CTCF is 12
n = set()
for pdb, uniacc in c2h2_zf_pdb2uniprot:
    if uniacc == CTCF and pdb.upper() in pdb_protein_dna:
        n.add(pdb)
print(len(n))

12


Finally, I extracted the "magic numbers"!!!

In [133]:
def overlap(start_A, end_A, start_B, end_B):
    """
    ---------AAAAAAA---------
    -----BBBBB--------------- = True
    ----------BBBBB---------- = True
    ---------------BBBBB----- = True
    ----BBBBB---------------- = False
    ----------------BBBBB---- = False
    """

    if start_A <= end_B and end_A >= start_B:

        return(True)

# The total number of C2H2-zf domains covered by a 3D structure in the PDB is?
n = set()
m = set()
pdbs = {}
for pdb, uniacc in c2h2_zf_pdb2uniprot:
    if uniacc in uprot2prosite:
        for domain in uprot2prosite[uniacc]:
            domain_start = int(domain[0])
            domain_end = int(domain[1])
            for pdb_chain in c2h2_zf_pdb2uniprot[(pdb, uniacc)]:
                pdb_chain_start = c2h2_zf_pdb2uniprot[(pdb, uniacc)][pdb_chain][0]
                pdb_chain_end = c2h2_zf_pdb2uniprot[(pdb, uniacc)][pdb_chain][-1]
                if overlap(domain_start, domain_end, pdb_chain_start, pdb_chain_end):
                    n.add(tuple([uniacc, domain]))
                    if pdb.upper() in pdb_protein_dna:
                        m.add(tuple([uniacc, domain]))
                        pdbs.setdefault(pdb.upper(), uniacc)
print("The total number of C2H2-zf domains covered by a 3D structure in the PDB is: %s" % len(n))
# The total number of human C2H2-zf domains covered by a 3D structure in the PDB is?
i = set()
for u, d in n:
    if u in human_seqs:
        i.add(tuple([u, d]))
print("For human, the number is: %s" % len(i))
# The total number of C2H2-zf domains covered by a protein-DNA complex in the PDB is?
print("Moreover, the total number of C2H2-zf domains covered by a protein-DNA complex in the PDB is: %s" % len(m))
# The total number of human C2H2-zf domains covered by a protein-DNA complex in the PDB is?
i = set()
for u, d in m:
    if u in human_seqs:
        i.add(tuple([u, d]))
print("And for human, the number is: %s" % len(i))

The total number of C2H2-zf domains covered by a 3D structure in the PDB is: 331
For human, the number is: 270
Moreover, the total number of C2H2-zf domains covered by a protein-DNA complex in the PDB is: 86
And for human, the number is: 48


In [134]:
# Must have list of PDB accessions (i.e. protein-DNA complexes of C2H2-zf domains)
print("The total number of protein-DNA complexes in the PDB covering one or more C2H2-zf domains is: %s" % len(pdbs))
print("For human the number is: %s" % len([p for p in pdbs if pdbs[p] in set(pdbs.values()).intersection(set(human_seqs.keys()))]))
print("The total number of C2H2-zf proteins with one or more domains covered by protein-DNA complexes in the PDB is: %s" % len(set(pdbs.values())))
print("For human, the number is: %s" % len(set(pdbs.values()).intersection(set(human_seqs.keys()))))
print()
print("Must have list of C2H2-zf protein-DNA complexes:")
for pdb in sorted(pdbs):
    print(pdb)

The total number of protein-DNA complexes in the PDB covering one or more C2H2-zf domains is: 104
For human the number is: 62
The total number of C2H2-zf proteins with one or more domains covered by protein-DNA complexes in the PDB is: 19
For human, the number is: 11

Must have list of C2H2-zf protein-DNA complexes:
1A1F
1A1G
1A1H
1A1I
1A1J
1A1K
1A1L
1AAY
1F2I
1G2D
1G2F
1JK1
1JK2
1LLM
1P47
1UBD
1ZAA
2GLI
2I13
2JP9
2JPA
2KMK
2LT7
2PRT
2WBS
2WBU
3UK3
4F2J
4F6M
4F6N
4GZN
4IS1
4M9E
4M9V
4R2A
4R2C
4R2D
4R2E
4R2P
4R2Q
4R2R
4R2S
4X9J
5EGB
5EH2
5EI9
5K5H
5K5I
5K5J
5K5L
5KE6
5KE7
5KE8
5KE9
5KEA
5KEB
5KKQ
5KL2
5KL3
5KL4
5KL5
5KL6
5KL7
5T00
5T0U
5UND
5V3J
5V3M
5VMU
5VMV
5VMW
5VMX
5VMY
5VMZ
5WJQ
5YEF
5YEG
5YEH
5YEL
5YJ3
6A57
6B0O
6B0P
6B0Q
6B0R
6BLW
6DF5
6DF8
6DF9
6DFA
6DFB
6DFC
6E93
6E94
6JNL
6JNM
6JNN
6KI6
6ML2
6ML3
6ML4
6ML5
6ML6
6ML7
